In [1]:
%pylab inline
%matplotlib inline

import pyprind
import os
import time
import collections
import codecs
from pprint import pprint
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

sys.path.append("/Users/Bya/git/predictEPL/utils/")
sys.path.append("/Users/Bya/git/predictEPL/config/")
sys.path.append("/Users/Bya/git/predictEPL/WebScrapping/")

import paths
import espn_urls
import my_plot
import scrap_espn_gamecast
import emolex
import useful_methods

Populating the interactive namespace from numpy and matplotlib


# Training a logistic regression model for document classification

In [2]:
from sklearn.grid_search import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
# generator function, 
# reads in and returns one document at a time
def stream_docs(path):
    with open(path, 'r') as csv:
        next(csv) # skip header
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label

In [5]:
next(stream_docs(path=paths.READ_PATH_REVIEW_SHORT + 'short_movie_reviews.csv'))

('"the rock is destined to be the 21st century\'s new "" conan "" and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal . "',
 1)

In [6]:
# will take a document stream from the stream_docs function
# return a particular number of documents specified by the size parameter
def get_minibatch(doc_stream, size):
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    return docs, y

In [8]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier

vect = HashingVectorizer(decode_error='ignore',
                        n_features=2**21,
                        preprocessor=None,
                        tokenizer=useful_methods.tokenizer)

clf = SGDClassifier(loss='log', random_state=1, n_iter=1)
doc_stream = stream_docs(path=paths.READ_PATH_REVIEW_SHORT + 'short_movie_reviews.csv')

### out-of-core learning

In [11]:
import pyprind
pbar = pyprind.ProgBar(45)
classes = np.array([0, 1])

for _ in range(45):
    X_train, y_train = get_minibatch(doc_stream, size=100)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)
    pbar.update()

0%                          100%
[                              ]

In [12]:
X_test, y_test = get_minibatch(doc_stream, size=5000)
X_test = vect.transform(X_test)
print('Accuracy: %.3f' % clf.score(X_test, y_test))

TypeError: 'NoneType' object is not iterable

In [ ]:
clf = clf.partial_fit(X_test, y_test)

In [ ]:
clf.score

In [ ]:
clf.intercept_

In [ ]:
clf.coef_

## Save the training model

In [ ]:
import pickle
import os

dest = os.path.join('movieclassifier', 'pkl_objects')
if not os.path.exists(dest):
    os.makedirs(dest)

pickle.dump(stop,
            open(os.path.join(dest, 'stopwords.pkl'),'wb'),
            protocol=4)

pickle.dump(clf,
            open(os.path.join(dest, 'classifier.pkl'), 'wb'),
            protocol=4)